# Caderno 1. Testes com BM25

Carrega as bases de questões e de pesquisa, indexa e calcula algumas métricas (precisão e recall).

In [1]:
import json
import pandas as pd
import os
from tqdm import tqdm
import pickle

In [2]:
REINDEXAR_INDICE_BM25_TODOS_CHUNKS = False
NOME_ARQUIVO_INDICE_BM25_TODOS_CHUNKS = 'outputs/1 - indices_invertidos/indice_bm25_todos_chunks.pickle'

REINDEXAR_INDICE_BM25_APENAS_ART = False
NOME_ARQUIVO_INDICE_BM25_APENAS_ART = 'outputs/1 - indices_invertidos/indice_bm25_apenas_art.pickle'

# 1. Carregar as bases de dados de chunks e de questões

Carrega as bases de dados de questões (são as queries) e os chunks de pesquisa (são os documentos a serem pesquisados).

In [3]:
def load_jsonl(path):
    with open(path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

chunks_pesquisa = load_jsonl('inputs/chunks_pesquisa.jsonl')
questoes = load_jsonl('inputs/questoes.jsonl')

## 1.1 Gera qrels no formato esperado considerando todos os chunks e no nível de artigo

O código para gerar as métricas considera um qrels no dataframe pandas. Gera o qrels no formato esperado pela ferramenta.

In [4]:
# Cria um qrels no formato esperado
id_questao = []
urn_chunk = []
score = []
rank = []
for q in questoes:
    total_docs = len(q['URN_FUNDAMENTACAO'])
    id_questao += [q['ID_QUESTAO']] * total_docs
    urn_chunk += q['URN_FUNDAMENTACAO']
    score += [1] * total_docs
    rank += list(range(1, total_docs+1))

qrels_todos_chunks = pd.DataFrame({
    "QUERY_KEY": id_questao,
    "DOC_KEY": urn_chunk,
    "SCORE": score,
    "RANK": rank
})

Filtra chunks_pesquisa para isolar apenas os chunks que são artigos completos. Isso apenas será utilizado na criação do índice, em outra seção.

In [5]:
chunks_pesquisa_apenas_art = [c for c in chunks_pesquisa if c['TIPO'] == 'ART' or c['TIPO'] == 'JUR']

Cria uma segunda lista de questões com o campo URN_FUNDAMENTACAO alterado para considerar apenas o nível de artigo.

In [6]:
import copy
import re

padrao = re.compile(r'!art\d{1,3}')

questoes_fund_apenas_art = copy.deepcopy(questoes)

for questao in questoes_fund_apenas_art:
    nova_fundamentacao = []

    for texto in questao.get("URN_FUNDAMENTACAO", []):
        match = padrao.search(texto)

        if match:
            # corta exatamente no final de !artX
            nova_fundamentacao.append(texto[:match.end()])
        else:
            nova_fundamentacao.append(texto)

    questao["URN_FUNDAMENTACAO"] = list(set(nova_fundamentacao))


Agora cria um qrels para essa situação uniformizada por artigo.

In [7]:
# Cria um qrels no formato esperado
id_questao = []
urn_chunk = []
score = []
rank = []
for q in questoes_fund_apenas_art:
    total_docs = len(q['URN_FUNDAMENTACAO'])
    id_questao += [q['ID_QUESTAO']] * total_docs
    urn_chunk += q['URN_FUNDAMENTACAO']
    score += [1] * total_docs
    rank += list(range(1, total_docs+1))

qrels_apenas_art = pd.DataFrame({
    "QUERY_KEY": id_questao,
    "DOC_KEY": urn_chunk,
    "SCORE": score,
    "RANK": rank
})

## 1.2 Mapa de URN -> CHUNK

Cria um mapa de urn -> chunk pra facilitar futuramente.

In [8]:
mapa_urn_chunk = { c['URN']: c for c in chunks_pesquisa}

# 2. Cria índices invertidos para o campo TEXTO

In [9]:
from bm25 import IndiceInvertido, BM25, tokenizador_pt_remove_html

# Índice para todos os chunks
iidx_todos_chunks = IndiceInvertido(tokenizador_pt_remove_html)
if REINDEXAR_INDICE_BM25_TODOS_CHUNKS or not os.path.exists(NOME_ARQUIVO_INDICE_BM25_TODOS_CHUNKS):
    # Se for indexar a primeira vez:
    # Demora cerca de 35 minutos para indexar
    iidx_todos_chunks.adiciona_objetos(chunks_pesquisa, lambda obj: (obj['URN'], obj['TEXTO']))
    iidx_todos_chunks.to_pickle(NOME_ARQUIVO_INDICE_BM25_TODOS_CHUNKS)
else:
    # Se quiser recuperar de um arquivo:
    iidx_todos_chunks.from_pickle(NOME_ARQUIVO_INDICE_BM25_TODOS_CHUNKS)

# Índice para os chunks apenas de artigos
iidx_apenas_art = IndiceInvertido(tokenizador_pt_remove_html)
if REINDEXAR_INDICE_BM25_APENAS_ART or not os.path.exists(NOME_ARQUIVO_INDICE_BM25_APENAS_ART):
    # Se for indexar a primeira vez:
    # Demora cerca de 35 minutos para indexar
    iidx_apenas_art.adiciona_objetos(chunks_pesquisa_apenas_art, lambda obj: (obj['URN'], obj['TEXTO']))
    iidx_apenas_art.to_pickle(NOME_ARQUIVO_INDICE_BM25_APENAS_ART)
else:
    # Se quiser recuperar de um arquivo:
    iidx_apenas_art.from_pickle(NOME_ARQUIVO_INDICE_BM25_APENAS_ART)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\P_8454\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\P_8454\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\P_8454\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


# 3. Pesquisa com BM25

Função auxiliar para, considerando uma lista ordenada de urns, selecionar as n primeiras urns considerando a regra de que nessa lista uma urn não pode englobar outra.

In [10]:
# Função para, dado uma lista de urns, selecionar as primeiras n urns.
# Para isso, é feita uma consideração de que uma urn pode englobar outra (vendo os atributos INICIO e FIM da urn).
# A lógica implementada é:
# 1. Começa a lista de resultados como vazia
# 2. Pega o próximo elemento de urns (a lista de chunks). Chamando NOVO
# 3. Antes de inserir na lista de resultados, verifica:
# 3.1. Se NOVO engloba algum elemento que está na lista, NOVO assume a posição do outro elemento, que sai da lista
# 3.2. Se NOVO é englobado por algum elemento que já está na lista, NOVO não entra na lista
# 3.3. Caso 3.1 ou 3.2 não ocorra, NOVO entra na lista
#
# Com isso, as n urns retornadas serão as que mais englobam contexto.
# Assim, é importante selecionar sempre o número que for usar no RAG.
# Por exemplo, se for utilizar só 3, o ideal é informar n=3. Se for utilizar só 5, informar n=5.
# Da forma como foi feita a implementação, informar n=5 e pegar os 3 primeiros resultados pode dar um resultado diferente 
# do que apenas informar n=3. Por exemplo, suponha que o resultado da lista é [art1_cpt_inc1, art2, art3, art1, art4, art6].
# Se usamos n=3, o resultado deverá ser [art1_cpt_inc1, art2, art3].
# No entanto, se usarmos n=5, o resultado deverá ser [art1, art2, art3, art4, art6], pois art1_cpt_inc1 tomou o lugar de art1.  
def seleciona_n_urns(urns, n):
    resultados = []

    for urn in urns:
        if len(resultados) >= n:
            break

        chunk_novo = mapa_urn_chunk[urn]
        ini_novo = chunk_novo['INICIO']
        fim_novo = chunk_novo['FIM']

        # EXCEÇÃO: INICIO == -1 → insere direto
        if ini_novo == -1:
            resultados.append(urn)
            continue
            
        indices_engloba = []
        descartar = False

        for i, urn_existente in enumerate(resultados):
            chunk_exist = mapa_urn_chunk[urn_existente]
            ini_exist = chunk_exist['INICIO']
            fim_exist = chunk_exist['FIM']

            # Caso 1: existente engloba novo → descarta
            if ini_exist <= ini_novo and fim_exist >= fim_novo:
                descartar = True
                break

            # Caso 2: novo engloba existente → marca para remoção
            if ini_novo <= ini_exist and fim_novo >= fim_exist:
                indices_engloba.append(i)

        if descartar:
            continue

        if indices_engloba:
            # remove do fim para o começo para não bagunçar índices
            for i in reversed(indices_engloba):
                del resultados[i]

            # insere na posição do primeiro removido
            resultados.insert(indices_engloba[0], urn)
        else:
            resultados.append(urn)

    return resultados

In [21]:
# Teste
seleciona_n_urns([
    "urn:lex:br:federal:constituicao:1988-10-05;1988!art1_par1u",
    "urn:lex:br:federal:constituicao:1988-10-05;1988!art1_cpt",
    "urn:lex:br:federal:constituicao:1988-10-05;1988!art1",
    "urn:lex:br:federal:constituicao:1988-10-05;1988!art2_cpt",
    "urn:lex:br:federal:constituicao:1988-10-05;1988!art3_cpt"
    ],3)

['urn:lex:br:federal:constituicao:1988-10-05;1988!art1',
 'urn:lex:br:federal:constituicao:1988-10-05;1988!art2_cpt',
 'urn:lex:br:federal:constituicao:1988-10-05;1988!art3_cpt']

In [23]:
# Teste
seleciona_n_urns(["tema_stf_483", "tema_stf_483"], 3)

['tema_stf_483', 'tema_stf_483']

# 3.1 Testes com BM25

In [12]:
# Agora instancia um BM25
buscador_todos_chunks = BM25(iidx_todos_chunks, k1=0.82, b=0.68, bias_idf=1)
buscador_apenas_art = BM25(iidx_apenas_art, k1=0.82, b=0.68, bias_idf=1)

# TODO: PRECISA TERMINAR DE IMPLEMENTAR A PARTE DE CHAMAR SELECIONA_N_URNS PARA QUE A PESQUISA USE A LÓGICA DE ENGLOBAR OS CHUNKS

In [13]:
def pesquisa_bm25(buscador, n_chunks=20):
    col_resultado_id_questao=[]
    col_resultado_urn_chunk=[]
    col_resultado_rank=[]
    
    for q in tqdm(questoes):
        id_questao = q['ID_QUESTAO']
        enunciado_com_alternativas = q['ENUNCIADO_COM_ALTERNATIVAS']
        resultados = buscador.pesquisar(enunciado_com_alternativas)

        # Resultados é uma lista de tuplas. Faz o unpack
        #urns, scores = zip(*resultados)
        #primeiros_n_chunks_urns = seleciona_n_urns(urns, n_chunks)
        
        primeiros_n_urns = [tupla_key_score[0] for tupla_key_score in resultados[:n_chunks]]
        ids_questao = [id_questao] * len(primeiros_n_urns)
        ranking = list(range(1, len(primeiros_n_urns)+1))
    
        col_resultado_id_questao.extend(ids_questao)
        col_resultado_urn_chunk.extend(primeiros_n_urns)
        col_resultado_rank.extend(ranking)
    
    df_resultados = pd.DataFrame({
        "QUERY_KEY": col_resultado_id_questao,
        "DOC_KEY": col_resultado_urn_chunk,
        "RANK": col_resultado_rank,
    })
    
    return df_resultados

In [14]:
k = 5
df_resultados_pesquisa_todos_chunks = pesquisa_bm25(buscador_todos_chunks, k)
df_resultados_pesquisa_apenas_art = pesquisa_bm25(buscador_apenas_art, k)

100%|████████████████████████████████████████████████████████████████████████████████| 700/700 [00:23<00:00, 29.21it/s]


In [15]:
from metricas import histograma_metricas, boxplot_metricas, metricas

df_metricas_pesquisa_todos_chunks = metricas(df_resultados_pesquisa_todos_chunks, qrels_todos_chunks, aproximacao_trec_eval=True, k=[k])
df_metricas_pesquisa_apenas_art = metricas(df_resultados_pesquisa_apenas_art, qrels_apenas_art, aproximacao_trec_eval=True, k=[k])

Resultados para a pesquisa com BM25 em todos os chunks

In [16]:
display(df_metricas_pesquisa_todos_chunks.describe())

,P@5,R@5,MRR@5,nDCG@5
count,700.000000,700.000000,700.000000,700.000000
mean,0.136857,0.396823,0.364714,0.324603
std,0.135581,0.429196,0.396257,0.356910
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.200000,0.211111,0.250000,0.208047
75%,0.200000,1.000000,0.500000,0.613147
max,0.600000,1.000000,1.000000,1.000000


In [17]:
display(df_metricas_pesquisa_apenas_art.describe())

,P@5,R@5,MRR@5,nDCG@5
count,700.000000,700.000000,700.000000,700.000000
mean,0.229143,0.790480,0.754024,0.723952
std,0.150752,0.357434,0.370664,0.347672
min,0.000000,0.000000,0.000000,0.000000
25%,0.200000,0.600000,0.500000,0.500000
50%,0.200000,1.000000,1.000000,1.000000
75%,0.200000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000


In [18]:
df_resultados_pesquisa_todos_chunks[df_resultados_pesquisa_todos_chunks.QUERY_KEY == '675'].iloc[0:k].DOC_KEY.tolist()

['tema_stf_483',
 'urn:lex:br:autoridade.nacional.protecao.dados;conselho.diretor:resolucao:2024-07-16;18;anexo.1!cap2_sec2',
 'urn:lex:br:autoridade.nacional.protecao.dados;conselho.diretor:resolucao:2024-07-16;18;anexo.1!art9',
 'urn:lex:br:autoridade.nacional.protecao.dados;conselho.diretor:resolucao:2024-07-16;18;anexo.1!cap2',
 'urn:lex:br:federal:lei:2011-11-18;12527!cap2']

In [19]:
df_resultados_pesquisa_apenas_art[df_resultados_pesquisa_apenas_art.QUERY_KEY == '675'].iloc[0:k].DOC_KEY.tolist()

['tema_stf_483',
 'urn:lex:br:autoridade.nacional.protecao.dados;conselho.diretor:resolucao:2024-07-16;18;anexo.1!art9',
 'urn:lex:br:federal:constituicao:1988-10-05;1988!art37',
 'urn:lex:br:federal:lei:1997-07-16;9472!art3',
 'urn:lex:br:federal:constituicao:1988-10-05;1988!art5']